#  A little introduction to Keras, using TensorFlow
    
 Keras is a simple and high-level definition interface 
 that make the use of TensorFlow more easier<br><br>

<font color=red size=2 ><B><i>!! This tutorial assumes that you have configured Keras
 to use the TensorFlow backend</font><B></i></br><br>





    
## Part One : Calling Keras layers on TensorFlow tensors

 __We will start with a simple example of MNIST digits classification.<br>
 We will build a TensorFlow digits classifier using a stack of Keras Dense layers__
 

***

We first have to create a Tensorflow session and registering it with Keras.<br>
* Keras will use this session to initialize all internal variables

In [1]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

Using TensorFlow backend.


We can now building our classifier exactly like we would do with TensorFlow

In [2]:
# placeholder that will contain our input digits, as flat vectors
img = tf.placeholder(tf.float32, shape=(None, 784))

 Keras layers can be called on TensorFlow tensors :<br><br>
 *Using Keras layers will speed up the model definition process*

In [3]:
from keras.layers import Dense

# Fully-connected layer with 128 unis
# and ReLU activation (Rectified Linear Unit)
x = Dense(128, activation='relu')(img)
x = Dense(128, activation='relu')(x)

# Output layer with 10 units and a softmax activation
preds = Dense(10, activation='softmax')(x)

We will define the placeholder for the labels and the loss function to use

In [4]:
labels = tf.placeholder(tf.float32, shape=(None, 10))

from keras.objectives import categorical_crossentropy
loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

Now the training part, with a TensorFlow optimizer :

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

#Initialization of all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

#Run training loop
with sess.as_default():
    for i in range(100):
        batch = mnist_data.train.next_batch(50)
        train_step.run(feed_dict={img: batch[0], labels: batch[1]})

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


Now we can evaluate the model :

In [6]:
from keras.metrics import categorical_accuracy as accuracy
acc_value = accuracy(labels, preds)
with sess.as_default():
    print (acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels}))

[ 1.  1.  1. ...,  1.  1.  1.]


Here we have used Keras as a syntaxical shortcut to generate an operator that just map some input tensors to some output tensors.
The optimization is done via a TensorFlow native optimizer rather than a Keras one and no Keras model have been used.<br><br>

Concerning the optimizers performances, there is little speed differences between Keras and TensorFlow use. Keras seems to be faster than TensorFlow nativ optimizer on most cases, but the differences is only from 5 to 10%.
So for most of the projects, it doesn't really matter which one you choose.



### Some different behaviors during training and testing phases

Some Keras layers (like Dropout or BatchNormalization) have different behaviors at training time and testing time.<br>
It's possible to know if a layer uses the "leaarning phase" (train/test) by printin the layer.uses_learning_phase variable. A boolean that is set to True if the layer has a different behavior on training mode and testin mode, and False otherwise.<br><br>

If the model use such layers, you need to specify the value of the learning phase as part of feed_dict. That way the model knows wether to apply the layer.<br><br>

The Keras learnin phase is accessible via the Keras backend. <br>
Like this :

In [7]:
from keras import backend as K
print (K.learning_phase())

Tensor("keras_learning_phase:0", dtype=bool)


To set the use of learning phase, write the value 1 for training mode or 0 for testing mode, to feed_dict :<br>
*(here we have added "session=sess" because run() need a default session to work)*

In [8]:
# train mode
train_step.run(feed_dict={img: batch[0], labels: batch[1],
                          K.learning_phase(): 1}, session=sess)

Here's the code to add the Dropout layer to our previous MNIST example :

In [9]:
from keras.layers import Dropout
from keras import backend as K

img = tf.placeholder(tf.float32, shape=(None, 784))
labels = tf.placeholder(tf.float32, shape=(None, 10))

x = Dense(128, activation='relu')(img)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(10, activation='softmax')(x)

loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Initialize all variables
init_op = tf.global_variables_initializer()

sess.run(init_op)
with sess.as_default():
    for i in range(100):
        batch = mnist_data.train.next_batch(50)
        train_step.run(feed_dict={img: batch[0],
                                  labels: batch[1],
                                  K.learning_phase(): 1})

acc_value = accuracy(labels, preds)
with sess.as_default():
    print (acc_value.eval(feed_dict={img: mnist_data.test.images,
                                    labels: mnist_data.test.labels,
                                    K.learning_phase(): 0}))

[ 1.  1.  1. ...,  1.  0.  1.]


!! *Nexts codes snipets are here to explain the usage. But if you execute them you need to include theses :*

In [10]:
from keras.layers import LSTM
import tensorflow as tf

## Compatibility with name scopes and device scopes
<br>
Keras layers and models are fully compatible with TensorFlow name scopes. For instance, consider the following code snippet:

In [11]:
x = tf.placeholder(tf.float32, shape=(None, 20, 64))
with tf.name_scope('block1'):
    y = LSTM(32, name='mylstm')(x)

The weights of our new LSTM layer will now be named block1/mylstm_W_i, block1/mylstm_U_i, and so on...

Similarly, devices scopes would work like this :


In [12]:
with tf.device('/gpu:0'):
    x = tf.placeholder(tf.float32, shape=(None, 20, 64))
    y = LSTM(32)(x)


## Compatibility with graph scopes

Any Keras layer or model that you define inside a TensorFlow graph scope will be created as part of the specified graph, with all he's variables an operations.

In [13]:
from keras.layers import LSTM
import tensorflow as tf

my_graph = tf.Graph()
with my_graph.as_default():
    x = tf.placeholder(tf.float32, shape=(None, 20, 64))
    # LSTM operators and variables are part of the Graph
    y = LSTM(32)(x)

## Compatibility with variables scopes

Variables sharing should be done via calling multiple instance of the same Keras layer, because the TenorFlow variable scope will have NO effect on a Keras model or layer.<br>
More informations on wheight sharing with Keras could be found on the "Wheight Sharing" section of the functional API guide.<br><br>

Here is an exemple of weight sharing by reusing the same layer instance :

In [14]:
# Instantiate a Keras layer
lstm = LSTM(32)

# Instantiate 2 TensorFlow placeholders
x = tf.placeholder(tf.float32, shape=(None, 20, 64))
y = tf.placeholder(tf.float32, shape=(None, 20, 64))

# Encoding the 2 tensors with the same LSTM weights
x_encoded = lstm(x)
y_encoded = lstm(y)

## Colleting trainable weights and state updates

